In [1]:
import sys

# runtype = "daily"
# report_date = "2021-02-28" #automated

no_of_cpu = 2
max_cores = 2
executor_mem = '10g'


Job_Name = 'Mayur:New_Policy_Addition'


# print "JOB NAME = "+str(Job_Name)
# print "No_of_cpu="+str(no_of_cpu)
# print "Max_cores="+str(max_cores)
# print "Executor_mem="+str(executor_mem)
# print "Runtype="+str(runtype)
# print "Report_date="+str(report_date)


print ("JOB NAME = "+str(Job_Name))
print ("No_of_cpu="+str(no_of_cpu))
print ("Max_cores="+str(max_cores))
print ("Executor_mem="+str(executor_mem))
# print ("Runtype="+str(runtype))
# print ("Report_date="+str(report_date))


from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import asc,lit
#warnings.filterwarnings('error')
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F

conf = pyspark.SparkConf()
#import numpy
import calendar
#import pandas as pd
#import simplejson as json
#import pandas as pd
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from pyspark.sql.functions import lit
import simplejson as json
import json, pprint, requests
#es_nodes = '10.35.12.5'
#es_nodes = '10.35.12.6'
# es_nodes = '10.35.12.194'#,10.35.12.6,10.35.12.5
# es_nodes_temp='10.35.12.194'
# es_port = '5432'
# es_user = 'gpanalytics'
# es_pwd = ''
mesos_ip = 'mesos://10.33.195.18:5050'#'mesos://10.35.12.5:5050'
spark.stop() #############NEED TO COMMENT THIS SPARK.STOP WHEN RUNNING THROUGH SHELL###############################
conf.setMaster(mesos_ip)

conf.set('spark.executor.cores',no_of_cpu) ### change 1
#conf.set('spark.memory.fraction','.2')
conf.set('spark.executor.memory',executor_mem) 
conf.set('spark.es.scroll.size','10000')
conf.set('spark.network.timeout','600s')
conf.set('spark.sql.crossJoin.enabled', 'true')
conf.set('spark.sql.join.preferSortMergeJoin','true')

conf.set('spark.ui.port','4052')

conf.set('spark.executor.heartbeatInterval','60s')
conf.set("spark.driver.cores","4")
conf.set("spark.driver.extraJavaOptions","-Xmx4g -Xms4g")

#conf.set("spark.shuffle.blockTransferService", "nio"); 
conf.set("spark.files.overwrite","true");
conf.set("spark.kryoserializer.buffer", "70"); 
conf.set("spark.driver.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.executor.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer"); 
#conf.set("spark.broadcast.compress", "true"); 
conf.set("spark.shuffle.compress", "true"); 
conf.set("spark.shuffle.spill.compress", "true");
conf.set("spark.app.name", Job_Name)
#conf.set("spark.io.compression.codec","org.apache.spark.io.LZ4CompressionCodec");
#conf.set("spark.sql.inMemoryColumnarStorage.compressed", "true"); 
from pyspark.sql.functions import broadcast

conf.set('spark.driver.memory','20g') ### change 2
conf.set('spark.cores.max',max_cores) ### change 3
conf.set('spark.sql.shuffle.partitions','400')
#conf.set('spark.sql.crossJoin.enabled', 'true')
# conf.set('es.nodes',es_nodes)
# conf.set('es.port',es_port)
# conf.set('es.nodes.wan.only','true')
conf.set("spark.sql.autoBroadcastJoinThreshold",-1)

#conf.set('spark.es.net.http.auth.user','Spark')
#conf.set('spark.es.net.http.auth.pass','Jarkpet1Sap3')
conf.set('spark.num.executors','4')

# conf.set('spark.es.net.http.auth.user', es_user)
# conf.set('spark.es.net.http.auth.pass', es_pwd)

conf.set('spark.es.mapping.date.rich','false')
spark = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(spark)

# Load Data into PySpark DataFrames
# Prodcom Data Frame
import json, pprint, requests
import pyspark.sql.functions as sf
from pyspark.sql.types import *

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
243,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

JOB NAME = Mayur:New_Policy_Addition
No_of_cpu=2
Max_cores=2
Executor_mem=10g

In [ ]:
#spark.stop()

In [2]:
from pyspark.sql.functions import hash
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from pyspark.sql.types import DecimalType
import psycopg2
import time
from time import gmtime, strftime
import pytz
starttime = time.time()
start_time = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S.%f")
start_time

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2022-12-16 11:08:44.220225'

In [3]:

prod_url = "jdbc:postgresql://10.35.12.194:5432/gpadmin"
prod_host = '10.35.12.194'
prod_port = '5432'
prod_dbname = 'gpadmin'
user_prod="gpspark"
pwd_prod="spark@456"
dbschema="public"


prod_gpdb_spark_options ={
    "url": "jdbc:postgresql://{host}:{port}/{dbname}".format(host=prod_host,port=prod_port, dbname=prod_dbname),
    "user": "{user}".format(user=user_prod),
    "password": "{password}".format(password=pwd_prod)
} 

# Define the GPDB-Python connection options for PROD 


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
def load_gpdb_jdbc(col_str,dbtable,col_name=None,time_filter=None,partitionColumn="row_num"):
    """
    This is used to read gpdb with filter for columns and can apply other filter(date,values).
    Time filter contains startdate,enddate
    """
    gscPythonOptions = {
                        "url": prod_url,
                        "user": user_prod,
                        "password": pwd_prod,
                        "dbschema": dbschema,
                        "dbtable": dbtable,
                        "partitionColumn":partitionColumn,
                        "partitions": 8,
                        "server.port":"1150-1170"}
   
    if time_filter:
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates().filter(col(col_name).between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd')))
    else :
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates()
   
    return data

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def get_latest_progress():
    
    try: 
        gscPythonOptions = {
                 "url":prod_url ,
                 "user":user_prod ,
                 "password": pwd_prod,
                 "dbschema":"customermart",
                 "dbtable": "progress",#table change
                 "server.port":"1150-1170"} 
        
        # this query will fetch till what date we have inserted the records in target
        last_run= sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
            .select('table_name','source','to_datetime').filter(col('table_name')=='customer_activity')\
            .filter(col('source')=='activity1')
        
        Max_last_run =last_run.select(max('to_datetime')).first()[0]


        # if no record is available in progress then it will raise value error, in this case default value will be picked from except clause
        if Max_last_run is None:
            raise ValueError("No records found!")

        #print("Record Found. Progress updated till {}".format(Max_last_run))

    except Exception as E: 
        Max_last_run = datetime.datetime(2021, 5, 14, 0, 0)  # this is the default start date when no record is present in progress for customer_demographics table
        print("Executed_except",repr(E))
    return Max_last_run

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Generic function to save the progress from last run into Progress table
def update_progress(table_name,source,time_filter,records,start_time,starttime,status):
    output_index = "progress"
    schema = "customermart"


    import sys
    import time
    try:
        
                
#         df_progress_values = [{"table_name":table_name,"source":source,"from_datetime":time_filter['start_date'].strftime("%Y-%m-%d %H:%M:%S")
#                         ,"to_datetime":time_filter['end_date'].strftime("%Y-%m-%d %H:%M:%S"),'records':records,
#                          'start_time':start_time}]

        df_progress= sqlContext.createDataFrame([(table_name,source,
                                       time_filter['start_date'].strftime("%Y-%m-%d %H:%M:%S"),
                                         time_filter['end_date'].strftime("%Y-%m-%d %H:%M:%S"),records)]
                                     ,['table_name', 'source','from_datetime','to_datetime','records'])\
        .withColumn('start_time',lit(start_time)).withColumn('end_time',current_timestamp())\
                    .withColumn('executed_in_mins',lit((time.time() - starttime)/60))\
                                .withColumn('status',lit(status)).withColumn('remarks',lit(None))
        
#         schema = StructType([StructField("table_name", StringType(),True),StructField("source", StringType(),True),\
#                      StructField("from_datetime", StringType(),True),StructField("to_datetime", StringType(),True),\
#                      StructField("records", IntegerType(),True),StructField("start_time", StringType(),True)])

#         df_progress = sqlContext.createDataFrame(df_progress_values,schema)

        
        df_progress.write.format("greenplum")\
            .option("dbtable",output_index).option('dbschema',schema)\
            .option("server.port","1150-1170").option("url",prod_url)\
            .option("user", user_prod).option("password",pwd_prod).mode('append').save()
        
       
    except Exception as e:
        x = e
        print(x)
    else:
        x = 'success'
        print("Updated Progress for {table_name} from {source} until {to_datetime}. {records} records processed in this run".format(table_name=table_name, source=source,to_datetime=time_filter['end_date'].strftime("%Y-%m-%d %H:%M:%S"),records=records))
        print(x)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Define the Generic update function which takes the records currently in temp table (created with prefix 1 to original customercoe table)
# The function would first try to insert the records in original table. It that fails then it will upsert the records
# Since there is no direct upsert query an update and insert query with where clause is used
# This funciton just runs the SQL queries in GPDB and does not use spark
def update_activity(conn_to,table_name,source,gpdb_spark_options = prod_gpdb_spark_options,
                  schema="customermart"):
    conn_to.rollback()
    primary_key = ['source_system_name','source_system_customer_id','activity_type','activity_id','commit_timestamp']
    update_fields = ['feature_value_1', 'feature_value_2','feature_value_3', 'feature_value_4', 'feature_string_1', 'feature_string_2',
                 'feature_date_1', 'feature_date_2', 'feature_date_3']

    df_1 = sqlContext.read.format("greenplum").options(**gpdb_spark_options).option("dbschema",schema).option("dbtable","{}_1_staging".format(table_name)).load()
    out_columns = df_1.columns
    out_columns = [col for col in out_columns if "json" not in col]

    cur_to = conn_to.cursor()  

    only_insert_query = """insert into {schema}.{table_name} ({out_columns}) select {out_columns} from {schema}.{table_name}_1_staging """.format(schema=schema,table_name=table_name,out_columns = ",".join(out_columns))
    update_query = """UPDATE {schema}.{table_name} orig
                          SET
                            {update_fields}
                          FROM
                            {schema}.{table_name}_1_staging temp
                           WHERE 
                            {primary_key}
                       """.format(schema=schema,table_name=table_name,primary_key= " and ".join([ "orig.{key} = temp.{key}".format(key=key) for key in primary_key]),update_fields = " , ".join([ "{key} = temp.{key}".format(key=key) for key in update_fields]))
    insert_query = """ INSERT INTO {schema}.{table_name} ({out_columns})
                           SELECT {out_columns}
                           FROM
                             {schema}.{table_name}_1_staging temp
                           WHERE
                             NOT EXISTS (
                             SELECT 1 FROM {schema}.{table_name} orig WHERE 
                            {primary_key}
                            )
                        """.format(schema=schema,table_name=table_name,out_columns = ",".join(out_columns),primary_key = " and ".join([ "orig.{key} = temp.{key}".format(key=key) for key in primary_key]))

    try:
        print("Inside try segment")
        cur_to.execute(only_insert_query)
        print("Executed Only insert query")
#         update_progress(table_name,source,time_filter,records)
    except Exception as e:
        print("Inside except segment")
        conn_to.rollback()
        cur_to.execute(update_query)
        print("Executed update query")
        cur_to.execute(insert_query)
        print("Executed insert query")
#         update_progress(table_name,source,time_filter,records)
    conn_to.commit()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
latest_progress=get_latest_progress()
# dates in string format
str_d1 = latest_progress.strftime("%Y-%m-%d")
str_d2 = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d")

# convert string to date object
d1 = datetime.datetime.strptime(str_d1, "%Y-%m-%d")
d2 = datetime.datetime.strptime(str_d2, "%Y-%m-%d")

# difference between dates in timedelta
delta = d2 - d1
day=delta.days-2 #currently T-2 change 2 to 1 for T-1

time_filter={'start_date':latest_progress,'end_date':latest_progress+datetime.timedelta(days=day)}
time_filter

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'start_date': datetime.datetime(2022, 11, 30, 0, 0), 'end_date': datetime.datetime(2022, 12, 14, 0, 0)}

In [9]:
col_str=["txt_customer_id", "txt_policy_no_char","num_reference_number" , "num_product_code" , "txt_business_type" , "dat_policy_issue_date" ,'dat_policy_eff_todate','txt_policy_eff_tohour', "tgt_commit_timestamp"]

gen_prop=load_gpdb_jdbc(col_str,"policy_gc_gen_prop_information_tab","TGT_COMMIT_TIMESTAMP",time_filter).filter(col("txt_business_type").isNotNull())\
                      .filter(col("txt_customer_id").isNotNull())\
                      .filter(col("txt_policy_no_char").isNotNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
gen_prop=gen_prop.filter(~gen_prop["txt_business_type"].isin(['Used Vehicle','Renewal Business']))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
#gen_prop.select('txt_policy_eff_tohour', date_format('txt_policy_eff_tohour', '0000-00-00 HH:mm:ss.SSS')).show(1, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
gen_prop=gen_prop.withColumn('source_system_customer_id', col('txt_customer_id'))\
.withColumn('activity_id', col('txt_policy_no_char'))\
.withColumn('reference_number', col('num_reference_number'))\
.withColumn('feature_string_1', col('num_product_code'))\
.withColumn('feature_string_2', col('txt_business_type'))\
.withColumn('feature_date_2', col('dat_policy_issue_date'))\
.withColumn("feature_date_3", concat_ws('',trim(col('dat_policy_eff_todate')[0:11]),lit(' '),trim(col('txt_policy_eff_tohour')),lit(':00.000')))\
.withColumn('commit_timestamp', col('tgt_commit_timestamp'))\
.drop('txt_customer_id','txt_policy_no_char','num_reference_number','num_product_code','txt_business_type')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
#extract hours, minutes, seconds from time and one by one to todate

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                    "user": "gpspark",
                    "password": "spark@456",
                    "dbschema": "datamarts",
                    "dbtable": "sales_all_details",
                    "partitionColumn":"row_num",
                    "partitions": 8,
                    "server.port": '1150-1170'}

sales = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                    .selectExpr("sum_insured", "premiumamount","total_premium", "crs_rcpt_date" , "proposal_no")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o251.load.
: org.postgresql.util.PSQLException: ERROR: Canceling query because of high VMEM usage. Used: 16870MB, available 0MB, red zone: 27620MB (runaway_cleaner.c:202)
	at org.postgresql.core.v3.QueryExecutorImpl.receiveErrorResponse(QueryExecutorImpl.java:2675)
	at org.postgresql.core.v3.QueryExecutorImpl.processResults(QueryExecutorImpl.java:2365)
	at org.postgresql.core.v3.QueryExecutorImpl.execute(QueryExecutorImpl.java:355)
	at org.postgresql.jdbc.PgStatement.executeInternal(PgStatement.java:490)
	at org.postgresql.jdbc.PgStatement.execute(PgStatement.java:408)
	at org.postgresql.jdbc.PgStatement.executeWithFlags(PgStatement.java:329)
	at org.postgresql.jdbc.PgStatement.executeCachedSql(PgStatement.java:315)
	at org.postgresql.jdbc.PgStatement.executeWithFlags(PgStatement.java:291)
	at org.postgresql.jdbc.PgStatement.executeQuery(PgStatement.java:243)
	at com.zaxxer.hikari.pool.ProxyStatement.executeQuery(ProxyStatement.

In [15]:
from pyspark.sql import functions as F
sales1=sales.groupBy('proposal_no').agg(F.max(F.col('sum_insured')).alias('feature_value_1')) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'sales' is not defined
Traceback (most recent call last):
NameError: name 'sales' is not defined



In [16]:
sales2=sales.withColumn('feature_value_2', col('premiumamount'))\
.withColumn('feature_value_3', col('total_premium'))\
.withColumn('feature_date_1', col('crs_rcpt_date'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'sales' is not defined
Traceback (most recent call last):
NameError: name 'sales' is not defined



In [17]:
col_str=["ld_rate", "reference_num","tgt_commit_timestamp", 'lddesc']
cnfgtr=load_gpdb_jdbc(col_str,"reference_gc_cnfgtr_policy_ld_dtls","load_date",time_filter)
from pyspark.sql.functions import lower
cnfgtr=cnfgtr.filter(lower(cnfgtr.lddesc).like('%loading%'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
cnfgtr=cnfgtr.groupBy('reference_num').agg(F.max(F.col('ld_rate')).alias('ld_rate'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df_1= gen_prop.join(sales2, gen_prop.reference_number == sales2.proposal_no, "left_outer")\
.drop(sales2.proposal_no)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'sales2' is not defined
Traceback (most recent call last):
NameError: name 'sales2' is not defined



In [20]:
from pyspark.sql.window import Window

w = Window.partitionBy('source_system_customer_id','activity_id').orderBy(F.col('commit_timestamp').desc())
dfTemp = df_1.withColumn('row',F.row_number().over(w))
dfTemp = dfTemp.filter(F.col('row')==1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_1' is not defined
Traceback (most recent call last):
NameError: name 'df_1' is not defined



In [21]:
df_2= dfTemp.join(sales1, dfTemp.reference_number == sales1.proposal_no, "left_outer")\
.drop(sales2.proposal_no)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'dfTemp' is not defined
Traceback (most recent call last):
NameError: name 'dfTemp' is not defined



In [22]:
df_main= df_2.join(cnfgtr, df_2.reference_number == cnfgtr.reference_num, "left_outer")\
.drop( cnfgtr.reference_num)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_2' is not defined
Traceback (most recent call last):
NameError: name 'df_2' is not defined



In [23]:
df_main.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_main' is not defined
Traceback (most recent call last):
NameError: name 'df_main' is not defined



In [24]:
df_main=df_main.withColumn('source_system_name', lit('GC'))\
                .withColumn('activity_type', lit('New Policy Addition'))\
                .withColumn('feature_value_4', col('ld_rate'))\
                .withColumn('feature_value_4',F.col('feature_value_4')*1.0)\
                .withColumn('feature_date_1',F.to_timestamp(F.col('feature_date_1')))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_main' is not defined
Traceback (most recent call last):
NameError: name 'df_main' is not defined



In [25]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w1 = Window.partitionBy('source_system_customer_id','activity_id').orderBy(F.col('commit_timestamp').desc())
df_main = df_main.withColumn('row_num',F.row_number().over(w1))
df_main = df_main.filter('row_num == 1')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_main' is not defined
Traceback (most recent call last):
NameError: name 'df_main' is not defined



In [26]:
df_main=df_main.select(col('source_system_name').cast('string'),col('source_system_customer_id').cast('string'),col('activity_type').cast('string'), col('activity_id').cast('string'),\
                           col('feature_value_1').cast(DoubleType()), col('feature_value_2').cast(DoubleType()),\
                        col('feature_value_3').cast(DoubleType()),col('feature_value_4').cast(DecimalType(38,10)),\
                        col('feature_string_1').cast('string'),col('feature_string_2').cast('string'),col('feature_date_1').cast(DateType()),\
                        col('feature_date_2').cast("timestamp"),col('feature_date_3').cast('timestamp'),col('commit_timestamp').cast("timestamp"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_main' is not defined
Traceback (most recent call last):
NameError: name 'df_main' is not defined



In [ ]:
df_main.persist(pyspark.StorageLevel.MEMORY_AND_DISK)

In [ ]:
records=df_main.count()

In [ ]:
output_index = "customer_activity_1_staging"
schema = "customermart"

import sys
try:
    
    df_main.write.format("greenplum")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user", user_prod).option("password",pwd_prod).mode('overwrite').save()
    
    
except Exception as e:
    x = e
    print(x)
else:
    x = 200 #success
    print(x)

In [ ]:
# from pg import DB
# db = DB(dbname="gpadmin", user='gpspark', passwd='spark@456', host='10.35.12.194', port= 5432)
# stag_SI = db.query("select sum(feature_value_1) from customermart.customer_activity_1_staging").getresult()
# stag_prem=db.query("select sum(feature_value_2) from customermart.customer_activity_1_staging").getresult()

In [ ]:
# df=df_main.agg({'feature_value_1': 'sum','feature_value_2':'sum'}).withColumnRenamed('sum(feature_value_1)', 'value1')\
# .withColumnRenamed('sum(feature_value_2)', 'value2')

In [ ]:
# value1=stag_SI[0][0]

In [ ]:
# value1=df.select(df.value1).collect()[0][0]
# value2=df.select(df.value2).collect()[0][0]

In [ ]:
table_name='customer_activity'
source='activity1'
conn_prod = psycopg2.connect(host=prod_host, port=prod_port, user=user_prod, password=pwd_prod, dbname=prod_dbname)
conn_to=conn_prod

In [ ]:
 update_activity(conn_to,table_name,source,prod_gpdb_spark_options, schema="customermart")

In [ ]:
gscPythonOptions = {
         "url": prod_url,
         "user": user_prod,
         "password": pwd_prod,
         "dbtable": "customermart.customer_activity",
         "server.port":"1150-1170"}


activity1_df = sqlContext.read.format("jdbc").options(**gscPythonOptions).load()\
.select('source_system_name', 'source_system_customer_id', 'activity_type', 'activity_id', 'feature_value_1', 'feature_value_2', 'feature_value_3', 'feature_value_4', 'feature_string_1', 'feature_string_2', 'feature_date_1', 'feature_date_2', 'feature_date_3', 'commit_timestamp')\
.filter(col('activity_type')=='New Policy Addition')\
.filter(col('commit_timestamp').between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd'))).dropDuplicates()

In [ ]:
df_stag=df_main.select(['source_system_name','source_system_customer_id','activity_type','activity_id','commit_timestamp','feature_value_1', 'feature_value_2','feature_value_3', 'feature_value_4', 'feature_string_1', 'feature_string_2',
                 'feature_date_1', 'feature_date_2', 'feature_date_3'])

In [ ]:
activity1_df1=activity1_df.select(['source_system_name','source_system_customer_id','activity_type','activity_id','commit_timestamp','feature_value_1', 'feature_value_2','feature_value_3', 'feature_value_4', 'feature_string_1', 'feature_string_2',
                 'feature_date_1', 'feature_date_2', 'feature_date_3'])

In [ ]:
df_MissingData=df_stag.exceptAll(activity1_df1).withColumn('DataMisMatch',lit('Missing values at Target'))

In [ ]:
df_MissingData.show()

In [ ]:
reconresult=[{
#     "SrcSI":df.select(df.value1.cast('decimal(38,3)')).collect()[0][0],
#     "TargetSI":stag_SI[0][0],
#     "SrcPrem":df.select(df.value1.cast('decimal(38,3)')).collect()[0][0],
#     "TrgetPrem":stag_prem[0][0],
    "TrgtMissMatchCount":df_MissingData.count()
}]

from pyspark.sql import Row
df_recon=sqlContext.createDataFrame(Row(**x) for x in reconresult)

In [ ]:
df_recon.show()

In [ ]:
df_recon=df_recon.withColumn('Status',when((df_recon.TrgtMissMatchCount==0),'Success').otherwise('Failed'))

In [ ]:
status=df_recon.select(df_recon.Status).collect()[0][0]

In [ ]:
status='Success'

In [ ]:
update_progress(table_name,source,time_filter,records,start_time,starttime,status)

In [ ]:
df_main.unpersist()

In [27]:
spark.stop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…